In [44]:
### Library Imports
from dash import Dash,html,dcc, Input, Output, State
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
import pathlib
import tiledbvcf
import shutil


In [54]:
# Dash PlayGround

# Instantiating the app
app = Dash("__name__", external_stylesheets=[dbc.themes.BOOTSTRAP] )

# CONSTANTS
FILES_DIRECTORY = ''
DATASET = 'vcf_dataset'

# App layout
app.layout = html.Div([
    dbc.Label("Path to VCF Files: "),
    dcc.Input(id = "files_directory", placeholder="Directory containing VCF files"),
    html.Br(),
    dbc.Label("Dataset Name: "),
    dcc.Input(id = "dataset", placeholder="Dataset Name ( Default : vcf_dataset )"),
    html.Button('Submit', id='constants_update_submit', n_clicks=0),
    html.P(id="Output")
])

@app.callback(Output("Output", component_property="children"),
          Input("constants_update_submit", component_property="n_clicks"),
          State("dataset", component_property="value"),
          State("files_directory", component_property="value"))

def update_output(n_clicks, dataset, files_directory):
    if n_clicks == 0:
        raise PreventUpdate
    FILES_DIRECTORY = pathlib.Path(files_directory)
    if FILES_DIRECTORY.exists() == False:
        return("File path doesn't exist")
    
    if len(dataset) == 0:
        DATASET = pathlib.Path("./vcf_dataset")
    else:
        DATASET = pathlib.Path(f"./{dataset}")
    
    if DATASET.exists():
        return(html.P("Dataset Already exist. Please select what you want to do next ?"),
        dcc.Dropdown(["Remove", "Update"], id="duplicate_dataset_case"),
        html.Button('Submit', id='ddbutton', n_clicks=0),
        html.P(id="output_message"))
    else:
        create_dataset(DATASET, FILES_DIRECTORY)
        print("Dataset Created")




def create_dataset(dataset_path, files_directory):
    ingested_files_count = 0
    files = list(files_directory.glob("*.vcf.gz"))
    dataset = tiledbvcf.Dataset(str(dataset_path),"w")
    dataset.create_dataset()
    print(f"The directory has {len(files)} vcf files")
    print("Files ingestion started")
    for file in files:
        try:
            dataset.ingest_samples([str(file)])
            ingested_files_count = ingested_files_count + 1
        except Exception as e:
            print(e)
    #print(f"{ingested_files_count} out of {len(files)} ingested")
    print("dataset returned")
    return(dataset)



@app.callback(Output("output_message", component_property="children"),
          Input("ddbutton", component_property="n_clicks"),
          State("duplicate_dataset_case", component_property="value"))
def check_dataset_existence(n_clicks,message):
    if n_clicks == 0:
       raise PreventUpdate
    else:
        if message=="Remove":
            shutil.rmtree(DATASET)
        else:
            dataset = tiledbvcf.Dataset(str(DATASET),"w")
            print(f"The directory has {len(files)} vcf files")
            print("Files ingestion started")
            for file in files:
                try:
                    dataset.ingest_samples([str(file)])
                    ingested_files_count = ingested_files_count + 1
                except Exception as e:
                    print(e)
            print("dataset returned")
            return(dataset)
            
    
    
    



# Running the app
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode = 'external', port=8052)
    
    
    

Dash app running on http://127.0.0.1:8052/
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[54], line 33, in update_output(
    n_clicks=1,
    dataset=None,
    files_directory='../Data/GIAB_vcfs'
)
     30 if FILES_DIRECTORY.exists() == False:
     31     return("File path doesn't exist")
---> 33 if len(dataset) == 0:
        dataset = None
     34     DATASET = pathlib.Path("./vcf_dataset")
     35 else:

TypeError: object of type 'NoneType' has no len()

The directory has 2 vcf files
Files ingestion started
dataset returned
Dataset Created


In [18]:
files_directory = pathlib.Path("../Data/dum")
dataset_path = pathlib.Path("./testa")
files = list(files_directory.glob("*.vcf.gz"))
dataset = tiledbvcf.Dataset(str(dataset_path),"w")
print(f"The directory has {len(files)} vcf files")
print("Files ingestion started")
for file in files:
    try:
        dataset.ingest_samples([str(file)])
        ingested_files_count = ingested_files_count + 1
    except Exception as e:
        print(e)
#print(f"{ingested_files_count} out of {len(files)} ingested")
print("dataset returned")
return(dataset)
    

The directory has 1 vcf files
Files ingestion started
name 'ingested_files_count' is not defined
dataset returned


SyntaxError: 'return' outside function (2384976202.py, line 15)

In [22]:

tiledbds = tiledbvcf.Dataset('testa','r')
print(f"The dataset has been created with {tiledbds.sample_count()} vcf samples.")

    

The dataset has been created with 3 vcf samples.


In [42]:

dataset_existence = html.Div([
    html.P("Dataset Already exist. Please select what you want to do next ?"),
    dcc.Dropdown(["Remove", "Update"], id="duplicate_dataset_case"),
    html.Button('Submit', id='ddbutton', n_clicks=0),
    html.P(id="Output")
])

@app.callback(Output("Output", component_property="children"),
          Input("ddbutton", component_property="n_clicks"),
          State("duplicate_dataset_case", component_property="value"))

def check_dataset_existence(n_clicks,message):
    if n_clicks == 0:
       raise PreventUpdate
    else:
        if message=="Remove":
            return("removed")
        else:
            return("Updated")
    
    
    

AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.